<a href="https://colab.research.google.com/github/QasimKhan5x/NLP-for-Business/blob/main/NLP_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

Download the data from google spreadsheets

In [2]:
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1lInyxuxoW8KDKOLmrmVRuEPSNDHWXOZ6mx0LDWPzCXc')
sheet = wb.worksheet('game_reviews')
data = sheet.get_all_values()

In [3]:
import numpy as np
import pandas as pd

In [35]:
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

,author age,recommended age,review
1,"Teen, 17 years old",age 7+,Kids dont listent to the parents saying ''ThIs...
2,"Kid, 11 years old",age 2+,I have been playing this game for many years a...
3,"Kid, 12 years old",age 7+,The game is great with no true inappropriate t...
4,"Teen, 13 years old",age 5+,Are you sure you got common sense. I would giv...
5,"Kid, 10 years old",age 5+,IDK WHAT TO SAY BUT DIS IS DA BEST GAME EVA


In [36]:
df['author age'].value_counts()

Teen, 13 years old    122
Kid, 11 years old      88
Kid, 10 years old      88
Kid, 12 years old      58
Kid, 9 years old       54
Teen, 14 years old     36
Teen, 17 years old     34
Teen, 16 years old     27
Teen, 15 years old     23
Kid, 8 years old       21
Kid, 7 years old        7
Kid, 6 years old        2
Kid, 5 years old        1
Name: author age, dtype: int64

Featurize the 'author age' column and then drop it

In [37]:
df['author_age_cat'] = df['author age'].str.split(', ', expand=True).loc[:, 0]
df['author_age'] = df['author age'].str.split(', ', expand=True).loc[:, 1].str.extract('(\d{1,2})')
df['author_age'] = pd.to_numeric(df['author_age'])
df.drop('author age', axis=1, inplace=True)
df.head()

,recommended age,review,author_age_cat,author_age
1,age 7+,Kids dont listent to the parents saying ''ThIs...,Teen,17
2,age 2+,I have been playing this game for many years a...,Kid,11
3,age 7+,The game is great with no true inappropriate t...,Kid,12
4,age 5+,Are you sure you got common sense. I would giv...,Teen,13
5,age 5+,IDK WHAT TO SAY BUT DIS IS DA BEST GAME EVA,Kid,10


Re-order the columns

In [38]:
cols = df.columns.tolist()
cols = cols[2:] + cols[:2]
df = df[cols]
df.head()

,author_age_cat,author_age,recommended age,review
1,Teen,17,age 7+,Kids dont listent to the parents saying ''ThIs...
2,Kid,11,age 2+,I have been playing this game for many years a...
3,Kid,12,age 7+,The game is great with no true inappropriate t...
4,Teen,13,age 5+,Are you sure you got common sense. I would giv...
5,Kid,10,age 5+,IDK WHAT TO SAY BUT DIS IS DA BEST GAME EVA


Classify review as safe if the recommended age is less than 10

In [70]:
rec_ages = pd.to_numeric(df['recommended age'].str.extract('(\d+)').loc[:, 0])
#df['safe'] = rec_ages.apply(lambda age: int(age < 10))
#df['safe'] = df['author_age_cat'].apply(lambda cat: int(cat == 'Kid'))
df['safe'] = df['author_age'].apply(lambda age: int(age < 10))
df.head()

,author_age_cat,author_age,recommended age,review,safe
1,Teen,17,age 7+,Kids dont listent to the parents saying ''ThIs...,0
2,Kid,11,age 2+,I have been playing this game for many years a...,0
3,Kid,12,age 7+,The game is great with no true inappropriate t...,0
4,Teen,13,age 5+,Are you sure you got common sense. I would giv...,0
5,Kid,10,age 5+,IDK WHAT TO SAY BUT DIS IS DA BEST GAME EVA,0


In [71]:
y = df['safe'].values

In [72]:
unique, counts = np.unique(y, return_counts=True)
unique, counts

(array([0, 1]), array([476,  85]))

In [73]:
reviews = df['review'].values

## NLP Part

In [74]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [75]:
import re

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer

import gensim
from gensim.parsing.preprocessing import remove_stopwords

# library that contains punctuation
import string

In [94]:
def remove_words(text):
    # remove url
    text = re.sub(r'http[^\s]+', '', text)
    # remove non word
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    # remove long words
    # text = re.sub(r'\w{25,}', '', text)
    # remove minecraft word
    text = re.sub(r'minecraft', '', text, flags=re.IGNORECASE)
    # remove multiple spaces
    return re.sub(r' +', ' ', text).strip()

In [95]:
punc = string.punctuation + '“”’━•'

# defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree= "".join([i for i in text if i not in punc])
    print([i for i in text if i not in punc])
    return punctuationfree

In [96]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

In [97]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemmatize(word):
    pos = get_wordnet_pos(word)
    return lemmatizer.lemmatize(word, pos)

def stem(word):
    return stemmer.stem(word)


def preprocess(text):
    result = []
    tokens = gensim.utils.simple_preprocess(text)
    for token in tokens:
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lemmatize(token))
    return ' '.join(result)

In [98]:
def clean_text(text):
    text = text.lower().strip()
    text = remove_words(text)
    text = remove_stopwords(text)
    # text = remove_punctuation(text)
    text = preprocess(text)
    return text

In [99]:
clean_text_vectorized = np.vectorize(clean_text)
clean_text_vectorized.__name__= "text cleaner"
reviews_preprocessed = clean_text_vectorized(reviews)

In [100]:
X_reviews = np.array([])
y_labels = np.array([])

for i, review in enumerate(reviews_preprocessed):
    if review != '':
        X_reviews = np.append(X_reviews, review)
        y_labels = np.append(y_labels, y[i])

In [101]:
X_reviews.shape, y_labels.shape

((524,), (524,))

In [102]:
(y_labels == 1).sum()

79

## ML Part

In [103]:
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.model_selection import KFold, RandomizedSearchCV

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [104]:
def crossvalidate_classifier(model, X, y, cm=False):
    
    scores = cross_val_score(model, X, y, scoring='f1_macro', cv=5)
    print("%0.2f f-1 score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
    y_pred = cross_val_predict(model, X, y, cv=5)
    
def evaluate_classifier(model, X_train, X_test, y_train, y_test):
      
    # ... fit your model here ...
    model.fit(X_train,y_train)

    # Run predict on your tfidf test data to get your predictions
    pred = model.predict(X_test)

    # Calculate your accuracy using the metrics module
    acc_score = metrics.accuracy_score(pred, y_test)
    print("Accuracy Score:   %0.3f" % acc_score)
    
    f1score = metrics.f1_score(pred, y_test, average='macro')
    print("F-1 Score:   %0.3f" % f1score)

    return model

def best_hyperparam(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=5, scoring_fit='f1_score',
                       do_probabilities = False):
    gs = RandomizedSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data, epochs=100)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [105]:
# Split training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_reviews, y_labels, 
                                                    random_state=42, 
                                                    test_size=0.25,
                                                    shuffle=True, 
                                                    stratify=y_labels)

In [106]:
# Initialize count vectorizer
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.9, min_df=0.05)

# Create count train and test variables
count_train = count_vectorizer.fit_transform(X_train)
count_test = count_vectorizer.transform(X_test)

# Initialize tfidf vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.9, min_df=0.05)

# Create tfidf train and test variables
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [107]:
print(count_train.shape, tfidf_train.shape)

(393, 89) (393, 89)


In [108]:
print(count_vectorizer.vocabulary_)

{'good': 27, 'game': 26, 'know': 34, 'let': 36, 'kid': 32, 'play': 55, 'people': 54, 'friend': 24, 'pc': 52, 'version': 79, 'dont': 19, 'think': 73, 'server': 65, 'way': 83, 'new': 47, 'fun': 25, 'building': 9, 'best': 4, 'child': 12, 'age': 0, 'little': 39, 'come': 13, 'survival': 68, 'mode': 43, 'online': 49, 'bad': 3, 'swear': 70, 'creativity': 15, 'great': 28, 'zombie': 88, 'die': 17, 'overall': 50, 'build': 8, 'want': 82, 'easy': 20, 'bit': 5, 'violence': 80, 'like': 38, 'lot': 40, 'violent': 81, 'kill': 33, 'animal': 2, 'survive': 69, 'creeper': 16, 'turn': 76, 'mob': 42, 'peaceful': 53, 'use': 78, 'buy': 10, 'chat': 11, 'multiplayer': 45, 'parent': 51, 'need': 46, 'thing': 72, 'learn': 35, 'food': 23, 'real': 60, 'life': 37, 'player': 57, 'world': 85, 'amaze': 1, 'young': 87, 'love': 41, 'time': 74, 'update': 77, 'pretty': 59, 'monster': 44, 'recommend': 61, 'creative': 14, 'year': 86, 'old': 48, 'different': 18, 'block': 6, 'sword': 71, 'blood': 7, 'say': 63, 'house': 30, 'scar

In [109]:
rf_bow = evaluate_classifier(RandomForestClassifier(), count_train, count_test, y_train, y_test)
rf_tfidf = evaluate_classifier(RandomForestClassifier(), tfidf_train, tfidf_test, y_train, y_test)

Accuracy Score:   0.840
F-1 Score:   0.456
Accuracy Score:   0.832
F-1 Score:   0.454


In [110]:
logreg_tfidf = evaluate_classifier(LogisticRegression(), tfidf_train, tfidf_test, y_train, y_test)

Accuracy Score:   0.847
F-1 Score:   0.459


In [111]:
crossvalidate_classifier(rf_tfidf, count_test, y_test)

0.45 f-1 score with a standard deviation of 0.01
